In [1]:
from __future__ import print_function
import numpy as np
from Grid import create_standard_grid
import copy

In [2]:
CONVERGENCE_THRESHOLD = 1e-3
ALL_POSSIBLE_ACTIONS = ('U', 'D', 'R', 'L')

In [55]:
def print_values(values, grid, title=None):
    if title:
        print(title)
        
    for i in reversed(range(grid.rows)):
        print('-'*(6*grid.cols+1))
        print('|', end="")
        for j in range(grid.cols):
            v = values.get((i,j), 0)
            if v > 0:
                print("+%.2f|" % v, end="")
            elif v < 0:
                print("%.2f|" % v, end="") 
            else:
                print(" 0.00|", end="")
        print("")
    print('-'*(6*grid.cols+1))

    
def print_policy(policies, grid, title=None):
    if title:
        print(title)
        
    for i in reversed(range(grid.rows)):
        print('-'*(4*grid.cols+1))
        print('|', end="")
        for j in range(grid.cols):
            action = policies.get((i,j), '.')
            print(" %s |" % action, end="") 
        print("")
    print('-'*(4*grid.cols+1))   

    
def generate_fixed_policy():
    """
    Return a fixed policy to win the game (using only one action for each state).
    This doesn't have to be unique.
    """
    policy = {
        (0, 0): 'R',
        (1, 0): 'U',
        (2, 0): 'R',
        (0, 1): 'R',
        (2, 1): 'R',
        (0, 2): 'U',
        (1, 2): 'U',
        (2, 2): 'R',
        (0, 3): 'L'
    }
    return policy

def generate_random_policy(grid):
    """
    Generate a random policy using available actions for each cell.
    """
    states = grid._actions.keys()
    policy_ = {}
    for st in states:
        policy_[st] = np.random.choice(grid._actions[st])
        
    return policy_

In [56]:
def run_policy_iteration_uniform_actions(grid, gamma_=1.0):
    """
    Run policy iterations assuming a uniform probability 
    of choosing possible actions at each cell.
    Return converged value function V (a dictionary of states (cell) and gain)
        V: {(i,j): value}
    """
    gamma = gamma_
    states = grid.get_all_states()
    # initialize Value functions
    V = {}
    for st in states:
        V[st] = 0
    
    count = 0
    # iterate until maximum deltaV is below a threshold.
    while True:
        biggest_delta_V = 0.0
        for st in states:
            old_v = V[st]
            states_with_possible_actions = grid._actions.keys()
            # go through all states that have possible actions. 
            # Essentially, exclude terminal states and dead cells.
            if st in states_with_possible_actions:
                new_v = 0
                actions = grid.get_actions(st)
                # each action has equal probability of being chosen
                prob_action = 1.0/len(actions)
                # go through all actions and update V[st]
                for act in actions:
                    # set the state back to st
                    grid.set_state(st)
                    reward = grid.move(act)
                    # state is now changed since we made a move based on a possible action
                    st_next = grid.get_current_state()
                    # update new_v for st
                    new_v += prob_action * (reward + gamma * V[st_next])
                V[st] = new_v
                # maximum different in successive iterations
                biggest_delta_V = max(biggest_delta_V, np.abs(old_v - new_v))
        count += 1
        if biggest_delta_V < CONVERGENCE_THRESHOLD:
            print("Policy iteration converged after {} iterations.".format(count))
            break
            
        if count % 5 == 0:
            print("    Iteration {} with deltaV {}".format(count, biggest_delta_V))
            
    return V


def run_fixed_policy(grid, fixed_policy=None, gamma_=0.9):
    """
    We assume a fixed policy is given (for actions at each state/cell).
    Iterate to find the value function for each state.
    Return converged value function V (a dictionary of states (cell), gain)
        V: {(i,j): value}
    """
    gamma = gamma_
    states = grid.get_all_states()
    # initialize Value functions
    V = {}
    for st in states:
        V[st] = 0

    count = 0
    while True:
        biggest_delta_V = 0.0
        for st in states:
            old_v = V[st]
            states_from_fixed_policy = fixed_policy.keys()
            # Go through all states that have possible actions. 
            # Essentially, exclude terminal states and dead cells.
            if st in states_from_fixed_policy:
                # set state tp current statu
                grid.set_state(st)
                # just one action per state (fixed policy)
                act = fixed_policy[st]
                reward = grid.move(act)
                # state is now changed since we made a move given action act.
                st_next = grid.get_current_state()
                # update V[st]
                new_v = reward + gamma * V[st_next]
                V[st] = new_v
                # find the biggest change (for convergence checking)
                biggest_delta_V = max(biggest_delta_V, np.abs(old_v - new_v))
        count += 1
        if biggest_delta_V < CONVERGENCE_THRESHOLD:
            print("Policy iteration converged after {} iterations.".format(count))
            break
        if count % 10 == 0:
            print("Iteration {} with deltaV {}".format(count, biggest_delta_V))
            
    return V

def run_random_policy_iteration_and_improvement(grid, policy_, gamma=0.9):
    """
    Iterate both on policy and value functions in two sub-steps:
    1- policy evaluation: Given current policy, find the value function for all states until V converges. 
    2- policy improvement: Given current policy/actions, find the best action resulting in best value function. 
    Convergence: When actions in two successive policy iterations don't change. 
    """
    def _initialize_V():
        """
        Initialize V[st]
        """
        V = {}
        states = grid.get_all_states()
        for st in states:
            if st in grid._actions.keys():
                V[st] = np.random.random()
            else: # terminal state
                V[st] = 0
                
        return V
    
    def _do_policy_evaluation():
        """
        Run one step of policy evaluation.
        Go through all states, pick action, move to next state, and update V using Bellman equation.
        Return the maximum deltaV (V_{k+1},V_k) amongst all states.
        """
        # get all states
        states = grid.get_all_states()
        delta_V_max = 0.0
        for st in states:
            old_v = V[st]
            policy_states = policy.keys()
            if st in policy_states:
                grid.set_state(st)
                act = policy[st]
                reward = grid.move(act)
                new_st = grid.get_current_state()
                new_v = reward + gamma * V[new_st]
                V[st] = new_v
                delta_V_max = max(delta_V_max, np.abs(old_v - new_v))
                
        return delta_V_max
    
    def _do_policy_improvement():
        """
        Go through all possible actions in current policy, find the 'best action'
        for state which V (value function) is maximum (gain).
        V is updated using Bellman equation. Upon convergence, return V,policy
        """
        def __find_best_action(st):
            """
            Find best action which maximizes value function V (via Bellman equation)
            """
            old_act = policy[st]
            best_act = None
            best_value = float('-inf')
            for act in grid.get_actions(st):
                grid.set_state(st)
                reward = grid.move(act)
                new_st = grid.get_current_state()
                v = reward + gamma * V[new_st]
                # find the action which yields in maximum value function
                if v > best_value:
                    best_value = v
                    best_act = act
                    
            return best_act
        
        states = grid.get_all_states()
        converged = True
        for st in states:
            # if state
            policy_states = policy.keys()
            if st in policy_states:
                old_act = policy[st]
                new_act = __find_best_action(st)
                policy[st] = new_act
                if old_act != new_act:
                    converged = False
                    return policy, converged
                
        # if we have reached here, policy is converged. 
        return policy, converged

    V = _initialize_V()
    policy = copy.deepcopy(policy_)
    count_1 = 0
    while True:
        count = 0
        # First, iterate on policy evaluation until convergence reached.
        while True:
            delta_V = _do_policy_evaluation()
            count += 1
            if delta_V < CONVERGENCE_THRESHOLD:
                print("   Policy evaluation converged in {} iterations with final error {}.".format(count, delta_V))
                break
            elif count % 5 == 0:
                pass
                # print("  Policy evaluation iteration: {}.".format(count))

        policy, is_policy_converged = _do_policy_improvement()
        count_1 += 1
        if is_policy_converged:
            print("Policy improvement converged in {} iterations.".format(count_1))
            break
        elif count_1 % 5 == 0:
            pass
            # print("  Policy improvement iteration: {}.".format(count_1))

    return policy, V

In [59]:
def run_experiment_uniform():
    print("Experiment 1: Uniform probablity value function iteration.")
    grid = create_standard_grid()
    V_uniform = run_policy_iteration_uniform_actions(grid, gamma_=1)
    print_values(V_uniform, grid, title="Value functions- Uniform probability.")
    print("*"*80)

In [61]:
def run_experiment_fixed_policy():
    print("Experiment 2: Fixed policy. Value function iteration")
    grid = create_standard_grid()
    fixed_policy = generate_fixed_policy()
    print_policy(fixed_policy, grid, title="Fixed (given) policy")
    V_fixed_policy = run_fixed_policy(grid, fixed_policy=fixed_policy, gamma_=0.9)
    print_values(V_fixed_policy, grid, title="Value function- Fixed policy")    
    print("*"*80)

In [64]:
def run_experiment_policy_evaluation_improvement():
    print("Experiment 3: Random policy w/ policy evaluation/improvement.")
    grid = create_standard_grid(step_negative_reward=-0.1)
    policy0 = generate_random_policy(grid)
    print_policy(policy0, grid, title="Random initial policy")
    policy_conv, V_conv = run_random_policy_iteration_and_improvement(grid, policy_=policy0, gamma=0.9)
    print_values(V_conv, grid, title="Converged value functions.")
    print_policy(policy_conv, grid, title="Converged policy.")

In [65]:
run_experiment_uniform()

Experiment 1: Uniform probablity value function iteration.
    Iteration 5 with deltaV 0.04093673792104857
    Iteration 10 with deltaV 0.011272908460062847
    Iteration 15 with deltaV 0.005267978380583682
    Iteration 20 with deltaV 0.002489071127286857
    Iteration 25 with deltaV 0.001176767267569323
Policy iteration converged after 27 iterations.
Value functions- Uniform probability.
-------------------------
|-0.03|+0.09|+0.22| 0.00|
-------------------------
|-0.16| 0.00|-0.44| 0.00|
-------------------------
|-0.29|-0.41|-0.54|-0.77|
-------------------------
********************************************************************************


In [66]:
run_experiment_fixed_policy()

Experiment 2: Fixed policy. Value function iteration
Fixed (given) policy
-----------------
| R | R | R | . |
-----------------
| U | . | U | . |
-----------------
| R | R | U | L |
-----------------
Policy iteration converged after 4 iterations.
Value function- Fixed policy
-------------------------
|+0.81|+0.90|+1.00| 0.00|
-------------------------
|+0.73| 0.00|+0.90| 0.00|
-------------------------
|+0.66|+0.73|+0.81|+0.73|
-------------------------
********************************************************************************


In [67]:
run_experiment_policy_evaluation_improvement()

Experiment 3: Random policy w/ policy evaluation/improvement.
Random initial policy
-----------------
| R | L | R | . |
-----------------
| U | . | U | . |
-----------------
| U | L | L | U |
-----------------
   Policy evaluation converged in 28 iterations with final error 0.0008669443650253239.
   Policy evaluation converged in 1 iterations with final error 0.0007022249356705146.
   Policy evaluation converged in 1 iterations with final error 0.0005688021978931257.
   Policy evaluation converged in 2 iterations with final error 0.0003731911220375972.
   Policy evaluation converged in 2 iterations with final error 0.0002448506951688856.
   Policy evaluation converged in 2 iterations with final error 0.0.
   Policy evaluation converged in 2 iterations with final error 0.0.
   Policy evaluation converged in 2 iterations with final error 0.0.
   Policy evaluation converged in 2 iterations with final error 0.0.
Policy improvement converged in 9 iterations.
Converged value functions.
-----